<a href="https://colab.research.google.com/github/onesummer123/c2s4_BDAO_Project/blob/main/BDAO_IMA_Checkin_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate and access Google Cloud services
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import the necessary libraries
from google.cloud import storage
from google.cloud import bigquery

# First make connection to BigQuery
# Replace 'your-project-id' with your Google Cloud project ID
# Replace 'your-dataset-name' with your dataset name
project_id = "c2s4-project"
dataset_name = "restaurant_dataset_1"
table_name = "restaurant_checkin" # if you make changes previously, then here you need to make according change

bigquery_client = bigquery.Client(project=project_id)

# Checkin Groupby ID

In [ ]:
# Query all data from the table
query = f"""
    SELECT *
    FROM `{project_id}.{dataset_name}.{table_name}`
"""

query_job = bigquery_client.query(query)
results = query_job.result()

checkin_df = query_job.to_dataframe()
checkin_df.head()

,business_id,date,total_number_of_checkin,restaurant,year,month,day,hour
0,7Vy1FHvdD22nzEB6zfaUFQ,2022-01-06 13:31:42+00:00,1028,True,2022,1,6,13
1,3RJbZk_yDY7D62Xt9t88IA,2022-01-04 18:06:55+00:00,518,True,2022,1,4,18
2,MQivsxEWhhdL8ARzO3jA8Q,2022-01-14 01:36:26+00:00,7,True,2022,1,14,1
3,cbG-YiFmhZ6AmGuuP6f64g,2022-01-02 16:52:50+00:00,7,True,2022,1,2,16
4,FGe8_MFciERzA_sM7FNJNA,2022-01-05 03:51:34+00:00,263,True,2022,1,5,3


In [ ]:
import pandas as pd

restaurant_value_counts = checkin_df['restaurant'].value_counts()

true_count = restaurant_value_counts[True] if True in restaurant_value_counts else 0
false_count = restaurant_value_counts[False] if False in restaurant_value_counts else 0

print("True：", true_count)
print("False：", false_count)

True： 8515582
False： 0


In [ ]:
# Query data from selected columns from the table
query = f"""
    SELECT business_id, total_number_of_checkin
    FROM `{project_id}.{dataset_name}.{table_name}`
"""

query_job = bigquery_client.query(query)
results = query_job.result()

checkin_by_id_df = query_job.to_dataframe()
checkin_by_id_df.head()

,business_id,total_number_of_checkin
0,4B5ORibJ3vhimhW8LTY6Fg,256
1,4B5ORibJ3vhimhW8LTY6Fg,256
2,4B5ORibJ3vhimhW8LTY6Fg,256
3,4zQV6v8TwEYMwI9Ekdf19g,256
4,52ov7d0148JvYG78_c-Bsw,256


In [ ]:
checkin_by_id_df = checkin_by_id_df.drop_duplicates(subset=['business_id'])
checkin_by_id_df.head()

,business_id,total_number_of_checkin
0,4B5ORibJ3vhimhW8LTY6Fg,256
3,4zQV6v8TwEYMwI9Ekdf19g,256
4,52ov7d0148JvYG78_c-Bsw,256
6,8PlUYO_9Q_c2ehidBhgvFg,256
8,B6Cn6maWQ6sLhLwYGLhSlg,256


In [ ]:
checkin_by_id_df.shape

(51318, 2)

In [ ]:
import pkgutil

# 检查是否安装了 pandas-gbq
if pkgutil.find_loader("pandas_gbq") is not None:
    print("pandas-gbq 已经安装")
else:
    print("pandas-gbq 未安装")

pandas-gbq 已经安装


In [ ]:
destination_table = 'restaurant_dataset_1.checkin_groupby_id_df'
project_id = 'c2s4-project'

# Use the to_gbq function to upload the DataFrame (after data processing) to BigQuery
checkin_by_id_df.to_gbq(destination_table, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 1302.98it/s]


# Checkin Groupby time

In [ ]:
project_id = "c2s4-project"
dataset_name = "restaurant_dataset_1"
table_name = "restaurant_checkin"
# Query data from selected columns from the table
query = f"""
    SELECT business_id, date, year, month, day, hour
    FROM `{project_id}.{dataset_name}.{table_name}`
"""

query_job = bigquery_client.query(query)
results = query_job.result()

checkin_by_time_df = query_job.to_dataframe()
checkin_by_time_df.head()

,business_id,date,year,month,day,hour
0,XDLVaR5WDrV5DRP4EXzcUQ,2011-10-01 17:28:10+00:00,2011,10,1,17
1,nX2105_S9lUhB5rbhC2n2w,2017-11-01 23:08:29+00:00,2017,11,1,23
2,yBrK-DwpdbJY2F795ZStcw,2014-10-01 14:23:19+00:00,2014,10,1,14
3,wHiPhPVuGnngEieDBMG00g,2013-06-01 17:56:51+00:00,2013,6,1,17
4,wHiPhPVuGnngEieDBMG00g,2016-10-01 22:32:26+00:00,2016,10,1,22


In [ ]:
checkin_by_time_df['checkin_number'] = 1
checkin_by_time_df.head()

,business_id,date,year,month,day,hour,checkin_number
0,XDLVaR5WDrV5DRP4EXzcUQ,2011-10-01 17:28:10+00:00,2011,10,1,17,1
1,nX2105_S9lUhB5rbhC2n2w,2017-11-01 23:08:29+00:00,2017,11,1,23,1
2,yBrK-DwpdbJY2F795ZStcw,2014-10-01 14:23:19+00:00,2014,10,1,14,1
3,wHiPhPVuGnngEieDBMG00g,2013-06-01 17:56:51+00:00,2013,6,1,17,1
4,wHiPhPVuGnngEieDBMG00g,2016-10-01 22:32:26+00:00,2016,10,1,22,1


In [ ]:
destination_table = 'restaurant_dataset_1.checkin_groupby_time_df'
project_id = 'c2s4-project'

# Use the to_gbq function to upload the DataFrame (after data processing) to BigQuery
checkin_by_time_df.to_gbq(destination_table, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 4500.33it/s]


In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

query = f"""
    SELECT
        business_id,name,state,city,longitude,latitude
    FROM
        `c2s4-project.restaurant_dataset_1.cleaned_restaurant_data_1`
    WHERE
        business_id IN (
            SELECT DISTINCT business_id FROM `c2s4-project.restaurant_dataset_1.checkin_groupby_time_df`
        )
"""
query_job = bigquery_client.query(query)
results = query_job.result()

result_df = query_job.to_dataframe()


merged_df = checkin_by_time_df.merge(result_df, on='business_id', how='left')

merged_df.head()

,business_id,date,year,month,day,hour,checkin_number,name,state,city,longitude,latitude
0,XDLVaR5WDrV5DRP4EXzcUQ,2011-10-01 17:28:10+00:00,2011,10,1,17,1,Fireside Brewhouse,IN,Greenwood,-86.084987,39.636183
1,nX2105_S9lUhB5rbhC2n2w,2017-11-01 23:08:29+00:00,2017,11,1,23,1,Donut Drive-In,MO,Saint Louis,-90.302837,38.592199
2,yBrK-DwpdbJY2F795ZStcw,2014-10-01 14:23:19+00:00,2014,10,1,14,1,Starbucks,NV,Sparks,-119.752917,39.555333
3,wHiPhPVuGnngEieDBMG00g,2013-06-01 17:56:51+00:00,2013,6,1,17,1,Mellow Mushroom - Brandon,FL,Brandon,-82.328670,27.922260
4,wHiPhPVuGnngEieDBMG00g,2016-10-01 22:32:26+00:00,2016,10,1,22,1,Mellow Mushroom - Brandon,FL,Brandon,-82.328670,27.922260


In [ ]:
# Check data types
merged_df.dtypes

business_id                    object
date              datetime64[us, UTC]
year                            Int64
month                           Int64
day                             Int64
hour                            Int64
checkin_number                  int64
name                           object
state                          object
city                           object
longitude                     float64
latitude                      float64
dtype: object

In [ ]:
# Check missing value
merged_df.isnull().sum()

business_id          0
date                 0
year                 0
month                0
day                  0
hour                 0
checkin_number       0
name              3190
state             3190
city              3190
longitude         3190
latitude          3190
dtype: int64

In [ ]:
merged_df = merged_df.dropna(subset=['name', 'state', 'city', 'longitude', 'latitude'], how='any')
merged_df.head()

,business_id,date,year,month,day,hour,checkin_number,name,state,city,longitude,latitude
0,XDLVaR5WDrV5DRP4EXzcUQ,2011-10-01 17:28:10+00:00,2011,10,1,17,1,Fireside Brewhouse,IN,Greenwood,-86.084987,39.636183
1,nX2105_S9lUhB5rbhC2n2w,2017-11-01 23:08:29+00:00,2017,11,1,23,1,Donut Drive-In,MO,Saint Louis,-90.302837,38.592199
2,yBrK-DwpdbJY2F795ZStcw,2014-10-01 14:23:19+00:00,2014,10,1,14,1,Starbucks,NV,Sparks,-119.752917,39.555333
3,wHiPhPVuGnngEieDBMG00g,2013-06-01 17:56:51+00:00,2013,6,1,17,1,Mellow Mushroom - Brandon,FL,Brandon,-82.328670,27.922260
4,wHiPhPVuGnngEieDBMG00g,2016-10-01 22:32:26+00:00,2016,10,1,22,1,Mellow Mushroom - Brandon,FL,Brandon,-82.328670,27.922260


In [ ]:
# Check missing value
merged_df.isnull().sum()

business_id       0
date              0
year              0
month             0
day               0
hour              0
checkin_number    0
name              0
state             0
city              0
longitude         0
latitude          0
dtype: int64

In [ ]:
destination_table = 'restaurant_dataset_1.merged_checkin_df'
project_id = 'c2s4-project'

# Use the to_gbq function to upload the DataFrame (after data processing) to BigQuery
merged_df.to_gbq(destination_table, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 905.12it/s]
